<a href="https://colab.research.google.com/github/FatiBuuloloo/Smart_Car_Recommender_SBERT-mini_project-007/blob/main/recommendation_system.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install sentence-transformers

In [ ]:
pip install openai

In [ ]:
import pandas as pd
import numpy as np
from sentence_transformers import SentenceTransformer, util
from IPython.display import Image, display
import warnings
warnings.filterwarnings("ignore")

In [ ]:
url = "https://raw.githubusercontent.com/FatiBuuloloo/Smart_Car_Recommender_SBERT-mini_project-007/main/BMW%20sales%20data%20(2010-2024).csv"
data = pd.read_csv(url)
data.head()

,Model,Year,Region,Color,Fuel_Type,Transmission,Engine_Size_L,Mileage_KM,Price_USD,Sales_Volume,Sales_Classification
0,5 Series,2016,Asia,Red,Petrol,Manual,3.5,151748,98740,8300,High
1,i8,2013,North America,Red,Hybrid,Automatic,1.6,121671,79219,3428,Low
2,5 Series,2022,North America,Blue,Petrol,Automatic,4.5,10991,113265,6994,Low
3,X3,2024,Middle East,Blue,Petrol,Automatic,1.7,27255,60971,4047,Low
4,7 Series,2020,South America,Black,Diesel,Manual,2.1,122131,49898,3080,Low


In [ ]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50000 entries, 0 to 49999
Data columns (total 11 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   Model                 50000 non-null  object 
 1   Year                  50000 non-null  int64  
 2   Region                50000 non-null  object 
 3   Color                 50000 non-null  object 
 4   Fuel_Type             50000 non-null  object 
 5   Transmission          50000 non-null  object 
 6   Engine_Size_L         50000 non-null  float64
 7   Mileage_KM            50000 non-null  int64  
 8   Price_USD             50000 non-null  int64  
 9   Sales_Volume          50000 non-null  int64  
 10  Sales_Classification  50000 non-null  object 
dtypes: float64(1), int64(4), object(6)
memory usage: 4.2+ MB


In [ ]:
for item in data.select_dtypes("object").columns:
  print(data[item].value_counts())
  print()

Model
7 Series    4666
i3          4618
i8          4606
3 Series    4595
5 Series    4592
X1          4570
X3          4497
X5          4487
M5          4478
X6          4478
M3          4413
Name: count, dtype: int64

Region
Asia             8454
Middle East      8373
North America    8335
Europe           8334
Africa           8253
South America    8251
Name: count, dtype: int64

Color
Red       8463
Silver    8350
Grey      8348
White     8304
Black     8273
Blue      8262
Name: count, dtype: int64

Fuel_Type
Hybrid      12716
Petrol      12550
Electric    12471
Diesel      12263
Name: count, dtype: int64

Transmission
Manual       25154
Automatic    24846
Name: count, dtype: int64

Sales_Classification
Low     34754
High    15246
Name: count, dtype: int64



In [ ]:
data.describe()

,Year,Engine_Size_L,Mileage_KM,Price_USD,Sales_Volume
count,50000.000000,50000.000000,50000.000000,50000.000000,50000.000000
mean,2017.015700,3.247180,100307.203140,75034.600900,5067.514680
std,4.324459,1.009078,57941.509344,25998.248882,2856.767125
min,2010.000000,1.500000,3.000000,30000.000000,100.000000
25%,2013.000000,2.400000,50178.000000,52434.750000,2588.000000
50%,2017.000000,3.200000,100388.500000,75011.500000,5087.000000
75%,2021.000000,4.100000,150630.250000,97628.250000,7537.250000
max,2024.000000,5.000000,199996.000000,119998.000000,9999.000000


In [ ]:
data.columns

Index(['Model', 'Year', 'Region', 'Color', 'Fuel_Type', 'Transmission',
       'Engine_Size_L', 'Mileage_KM', 'Price_USD', 'Sales_Volume',
       'Sales_Classification'],
      dtype='object')

In [ ]:
def tag_price(price):
    if price <60000:
        return "Afforable Car"
    elif 60000<=price<90000:
        return "Mid-Range Car"
    else:
        return "Expensive Car"
data["Tag_Year"] = data["Year"].apply(lambda x: "Old Type Car" if x<2020 else "New Type Car")
data["Tag_Mileage"] = data["Mileage_KM"].apply(lambda x: "Low Mileage" if x<=25000 else "High Mileage")
data["Tag_Efficiency"] = data["Engine_Size_L"].apply(lambda x: "Efficient with small engine" if x<=2 else "Inefficient with big engine")
data["Sales_Classification"] = data["Sales_Classification"].apply(lambda x: x+" Sale")
data["Tag_price"] = data["Price_USD"].apply(tag_price)

In [ ]:
data

,Model,Year,Region,Color,Fuel_Type,Transmission,Engine_Size_L,Mileage_KM,Price_USD,Sales_Volume,Sales_Classification,Tag_Year,Tag_Mileage,Tag_Efficiency,Tag_price
0,5 Series,2016,Asia,Red,Petrol,Manual,3.5,151748,98740,8300,High Sale,Old Type Car,High Mileage,Inefficient with big engine,Expensive Car
1,i8,2013,North America,Red,Hybrid,Automatic,1.6,121671,79219,3428,Low Sale,Old Type Car,High Mileage,Efficient with small engine,Mid-Range Car
2,5 Series,2022,North America,Blue,Petrol,Automatic,4.5,10991,113265,6994,Low Sale,New Type Car,Low Mileage,Inefficient with big engine,Expensive Car
3,X3,2024,Middle East,Blue,Petrol,Automatic,1.7,27255,60971,4047,Low Sale,New Type Car,High Mileage,Efficient with small engine,Mid-Range Car
4,7 Series,2020,South America,Black,Diesel,Manual,2.1,122131,49898,3080,Low Sale,New Type Car,High Mileage,Inefficient with big engine,Afforable Car
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
49995,i3,2014,Asia,Red,Hybrid,Manual,4.6,151030,42932,8182,High Sale,Old Type Car,High Mileage,Inefficient with big engine,Afforable Car
49996,i3,2023,Middle East,Silver,Electric,Manual,4.2,147396,48714,9816,High Sale,New Type Car,High Mileage,Inefficient with big engine,Afforable Car
49997,5 Series,2010,Middle East,Red,Petrol,Automatic,4.5,174939,46126,8280,High Sale,Old Type Car,High Mileage,Inefficient with big engine,Afforable Car
49998,i3,2020,Asia,White,Electric,Automatic,3.8,3379,58566,9486,High Sale,New Type Car,Low Mileage,Inefficient with big engine,Afforable Car


In [ ]:
data.columns

Index(['Model', 'Year', 'Region', 'Color', 'Fuel_Type', 'Transmission',
       'Engine_Size_L', 'Mileage_KM', 'Price_USD', 'Sales_Volume',
       'Sales_Classification', 'Tag_Year', 'Tag_Mileage', 'Tag_Efficiency',
       'Tag_price'],
      dtype='object')

In [ ]:
for item in data.select_dtypes("object").columns:
  print(data[item].value_counts())
  print()

Model
7 Series    4666
i3          4618
i8          4606
3 Series    4595
5 Series    4592
X1          4570
X3          4497
X5          4487
M5          4478
X6          4478
M3          4413
Name: count, dtype: int64

Region
Asia             8454
Middle East      8373
North America    8335
Europe           8334
Africa           8253
South America    8251
Name: count, dtype: int64

Color
Red       8463
Silver    8350
Grey      8348
White     8304
Black     8273
Blue      8262
Name: count, dtype: int64

Fuel_Type
Hybrid      12716
Petrol      12550
Electric    12471
Diesel      12263
Name: count, dtype: int64

Transmission
Manual       25154
Automatic    24846
Name: count, dtype: int64

Sales_Classification
Low Sale     34754
High Sale    15246
Name: count, dtype: int64

Tag_Year
Old Type Car    33299
New Type Car    16701
Name: count, dtype: int64

Tag_Mileage
High Mileage    43706
Low Mileage      6294
Name: count, dtype: int64

Tag_Efficiency
Inefficient with big engine    42130
Eff

In [ ]:
def make_metadata(row):
    model = row['Model']
    year = row['Year']
    region = row['Region']
    color = row['Color']
    fuel = row['Fuel_Type']
    trans = row['Transmission']
    engine = row['Engine_Size_L']
    mileage = row['Mileage_KM']
    price = row['Price_USD']
    sales_vol = row['Sales_Volume']
    sales_class = row['Sales_Classification']
    tag_year = row['Tag_Year']
    tag_mileage = row['Tag_Mileage']
    tag_efficiency = row['Tag_Efficiency']
    tag_price = row['Tag_price']
    description = (
        f"{tag_year} {model} in {color}. "
        f"This {tag_price} features a {trans} {fuel} engine. "
        f"Located in {region}, it has an engine size of {engine}L. "
        f"Condition: {tag_mileage} with {mileage} km. "
        f"Performance-wise, it is an {tag_efficiency}. "
        f"Market Status: {sales_class} ({sales_vol} units sold) at ${price}."
    )

    return description
data['Metadata'] = data.apply(make_metadata, axis=1)
data["Metadata"][0]

'Old Type Car 5 Series in Red. This Expensive Car features a Manual Petrol engine. Located in Asia, it has an engine size of 3.5L. Condition: High Mileage with 151748 km. Performance-wise, it is an Inefficient with big engine. Market Status: High Sale (8300 units sold) at $98740.'

In [ ]:
data["Metadata"] = data["Metadata"].apply(lambda x: x.lower())

In [ ]:
data["Metadata"][0]

'old type car 5 series in red. this expensive car features a manual petrol engine. located in asia, it has an engine size of 3.5l. condition: high mileage with 151748 km. performance-wise, it is an inefficient with big engine. market status: high sale (8300 units sold) at $98740.'

In [ ]:
model = SentenceTransformer('all-mpnet-base-v2')
sentences_embeddings = model.encode(data["Metadata"])

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/571 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [ ]:
CONSTRAINTS = {
    "below": ["under", "below", "less than", "lower than", "max", "maximum", "at most", "up to", "cheaper"],
    "above": ["over", "above", "more than", "greater than", "min", "minimum", "at least", "starting from"],
    "around": ["around", "about", "approximately", "near", "roughly", "in"]
}
COLUMNS = {
    "Price_USD": ["price", "cost", "budget", "$", "k"],
    "Year": ["year", "model", "made in"],
    "Engine_Size_L": ["engine", "liter", "liters", "l", "cc"],
    "Mileage_KM": ["mileage", "km", "miles", "driven"]
}
def extract_filters(user_input):
    filters_to_apply = []
    text = user_input.lower()
    text = re.sub(r'(\d+)k\b', r'\g<1>000', text)
    matches = list(re.finditer(r'\b\d+(?:\.\d+)?\b', text))

    for match in matches:
        value = float(match.group())
        start_idx = match.start()
        end_idx = match.end()
        context_before = text[max(0, start_idx-25) : start_idx]
        context_after = text[end_idx : min(len(text), end_idx+15)]
        full_context = context_before + " " + context_after
        detected_column = None
        for col, keywords in COLUMNS.items():
            if any(kw in full_context for kw in keywords):
                detected_column = col
                break
        if not detected_column:
            if 2009 <= value <= 2026: detected_column = "Year"
            elif value >= 1000: detected_column = "Price_USD"
            elif value < 10: detected_column = "Engine_Size_L"
        detected_logic = "around"
        min_dist = float('inf')

        for logic, keywords in CONSTRAINTS.items():
            for kw in keywords:
                if kw in context_before:
                    dist = start_idx - (text.find(kw, max(0, start_idx-25)) + len(kw))
                    if dist < min_dist:
                        min_dist = dist
                        detected_logic = logic
                elif kw in context_after:
                    dist = text.find(kw, end_idx) - end_idx
                    if dist < min_dist:
                        min_dist = dist
                        detected_logic = logic
        if detected_column:
            filters_to_apply.append({
                "column": detected_column,
                "logic": detected_logic,
                "value": value
            })

    return filters_to_apply

def filter_data(df, logic, value, column):
    if logic == "below":
        df = df[df[column] < value]
    elif logic == "above":
        df = df[df[column] > value]
    elif logic == "around":
        if column == "Year":
            df = df[df["Year"].astype(int) == int(value)]
        else:
            threshold = value * 0.1
            df = df[df[column].between(value - threshold, value + threshold)]
    return df

In [ ]:
def get_github_image_url(model_name):
    base_url = "https://raw.githubusercontent.com/FatiBuuloloo/Smart_Car_Recommender_SBERT-mini_project-007/main/BMW_Pictures/"
    extension_map = {
        "3 Series": ".png", "5 Series": ".png", "7 Series": ".jpg",
        "M3": ".png", "M5": ".jpg", "X1": ".png", "X3": ".jpg",
        "X5": ".jpg", "X6": ".jpg", "i3": ".jpg", "i8": ".jpg"
    }

    ext = extension_map.get(model_name, ".jpg")
    formatted_name = model_name.replace(" ", "%20")

    return f"{base_url}{formatted_name}{ext}"

In [48]:
def recom_final (user_input, sentences_embeddings):
    filters = extract_filters(user_input)
    data_copy = data.copy()
    filtered_indices = data_copy.index.tolist()
    for i in range(len(filters)):
        data_copy = filter_data(
            data_copy,
            filters[i]["logic"],
            filters[i]["value"],
            filters[i]["column"])
    if not filtered_indices:
        print("No Result Found!")
    else:
        filtered_embeddings = sentences_embeddings[filtered_indices]
        user_vector = model.encode(user_input, convert_to_tensor=True)
        similarities = util.cos_sim(user_vector, filtered_embeddings)
        k = min(5, len(filtered_indices))
        top_results = torch.topk(similarities[0], k=k)

        scores = top_results.values
        local_indices = top_results.indices

        print(f"Top {k} Recommendation:")
        for score, local_idx in zip(scores, local_indices):
            original_idx = filtered_indices[local_idx.item()]
            row = data.iloc[original_idx]
            img_url = get_github_image_url(row['Model'])

            print(f"{row['Model']} Model")
            try:
                display(Image(url=img_url, width=300))
            except:
                print("[No Picture]")

            print(f"Score: {score:.4f}")
            print(f"Year: {row['Year']} \nPrice: ${row['Price_USD']}")
            print(f"Engine: {row['Engine_Size_L']}L \nMileage: {row['Mileage_KM']}Km")
            print()

In [ ]:
user_input = "a car with big engine size with price below 100k in year above 2020"
recom_final(user_input, sentences_embeddings)